In [1]:
import numpy as np
import pandas as pd
import sys,os
import random
import copy
from time import time

import matplotlib.pyplot as plt
import seaborn as sns

from utils.method import read_bic_table

from utils.eval import make_ref_groups
from utils.eval import calculate_perfromance, compare_gene_clusters

In [2]:
exprs_file_t = "data/preprocessed_v6/TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv"
basename_t = "TCGA"

exprs_file_m = "data/preprocessed_v6/METABRIC_1904_17Kgenes.log2_exprs_z_v6.tsv"
basename_m = "METABRIC" 

m_subtypes = pd.read_csv("data/preprocessed_v6/METABRIC_1904_17Kgenes.subtypes_and_signatures_v6.tsv",sep = "\t",index_col=0)
m_annotation = pd.read_csv("data/preprocessed_v6/METABRIC_1904.annotation_v6.tsv",sep = "\t",index_col=0)

t_subtypes = pd.read_csv("data/preprocessed_v6/TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv",sep = "\t",index_col=0)
t_annotation = pd.read_csv("data/preprocessed_v6/TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv",sep = "\t",index_col=0)


exprs_t= pd.read_csv(exprs_file_t,sep = "\t",index_col=0)
exprs_t[exprs_t>3] = 3
exprs_t[exprs_t<-3] = -3

exprs_m= pd.read_csv(exprs_file_m,sep = "\t",index_col=0)
exprs_m[exprs_m>3] = 3
exprs_m[exprs_m<-3] = -3

In [3]:
known_groups_t, all_samples_t = make_ref_groups(t_subtypes, t_annotation,exprs_t)
known_groups_m, all_samples_m = make_ref_groups(m_subtypes, m_annotation,exprs_m)

In [4]:
n_runs = 5
seeds = []
random.seed(42)
for i in range(n_runs):
    seeds.append(random.randint(0,1000000))
print("generate ",n_runs," seeds",seeds)

generate  5  seeds [670487, 116739, 26225, 777572, 288389]


In [5]:
subt_t = []
subt_m = []
clustering_similarities = []

pvals = [0.0001, 0.0005,0.005,0.001,0.01,0.05]
bin_methods = ["kmeans","GMM","ward"] 

In [6]:
from run_unpast import run

In [7]:
### Louvain 
out_dir= "results_on_real_data/"
modularities = [0,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

subt_t = []
subt_m = []
clustering_similarities = []


for pval in pvals:
    for bin_method in bin_methods:
        for m in modularities:
            # save parameters as a ;-separated string
            params = "bin="+bin_method+";pval="+str(pval)
            params += ";clust="+"Louvain"+";m="+str(m)
            print()
            for r in range(n_runs):
                seed = seeds[r]
                params_dict = {"parameters":params, "seed":seed,"run":r}
                ### running TCGA or reading results
                try:
                    t0 = time()
                    fname = out_dir+basename_t+".seed="+str(seed)+\
                    ".bin="+bin_method +",pval="+str(pval)+",clust=Louvain"+",m="+str(m)+".biclusters.tsv"
                    result_t = read_bic_table(fname)
                    """result_t = run_DESMOND(exprs_file_t, basename_t, out_dir=out_dir,
                                                save=True, load = True,
                                                ceiling = 3,
                                                min_n_samples = 5,
                                                bin_method = bin_method, pval = pval,
                                                clust_method = "Louvain",
                                                similarity_cutoffs = similarity_cutoffs,
                                                seed = seed,
                                                verbose = False)"""
                    time_t = time()-t0
                    # find the best matches between TCGA biclusters and subtypes
                    # and calculate overall performance == weighted sum of Jaccard indexes
                    performance_t,bm_dict_t = calculate_perfromance(result_t, known_groups_t,all_samples_t)
                    performance_t = performance_t.to_dict()
                    performance_t.update(params_dict)
                    performance_t["time"] = time_t
                    subt_t.append(performance_t)
                    t_failed = False
                except:
                    print("TCGA biclustering failed with ",seed,  pval,bin_method ,file = sys.stderr)
                    print(fname)
                    t_failed = True
                    subt_t.append({params_dict})
                
                ### running METABRIC or reading results
                try:
                    t0 = time()
                    fname = out_dir+basename_m+".seed="+str(seed)+\
                    ".bin="+bin_method +",pval="+str(pval)+",clust=Louvain"+",m="+str(m)+".biclusters.tsv"
                    result_m = read_bic_table(fname)
                    """result_m = run_DESMOND(exprs_file_m, basename_m, out_dir=out_dir,
                                                save=True, load = True,
                                                ceiling = 3,
                                                min_n_samples = 5,
                                                bin_method = bin_method, pval = pval,
                                                clust_method = "Louvain",
                                                similarity_cutoffs = similarity_cutoffs,
                                                seed = seed,
                                                verbose = False)"""
                    time_m = time()-t0
                    # find the best matches between METABRIC biclusters and subtypes
                    # and calculate overall performance == weighted sum of Jaccard indexes
                    performance_m,bm_dict_m = calculate_perfromance(result_m, known_groups_m,all_samples_m)
                    performance_m = performance_m.to_dict()
                    performance_m.update(params_dict)
                    performance_m["time"] = time_m
                    subt_m.append(performance_m)
                    m_failed = False
                except:
                    print("METABRIC biclustering failed with ",seed,  pval,bin_method ,file = sys.stderr)
                    print(fname)
                    m_failed = True
                    subt_m.append(params_dict)
                print(params,seed, round(performance_t["PAM50"],3),round(performance_m["PAM50"],3))    
                # compare clustering results - only if gene sets are defined for each cluster
                if not (t_failed or m_failed): 
                    N = exprs_m.shape[0]
                    clust_sim, bm, bm2 = compare_gene_clusters(result_t,result_m, N)                    
                else:
                    clust_sim = {}
                clust_sim.update(params_dict)
                clustering_similarities.append(clust_sim)
                               


bin=kmeans;pval=0.0001;clust=Louvain;m=0 670487 0.842 0.833
bin=kmeans;pval=0.0001;clust=Louvain;m=0 116739 0.848 0.833
bin=kmeans;pval=0.0001;clust=Louvain;m=0 26225 0.848 0.838
bin=kmeans;pval=0.0001;clust=Louvain;m=0 777572 0.842 0.838
bin=kmeans;pval=0.0001;clust=Louvain;m=0 288389 0.847 0.838

bin=kmeans;pval=0.0001;clust=Louvain;m=0.3 670487 0.832 0.801
bin=kmeans;pval=0.0001;clust=Louvain;m=0.3 116739 0.838 0.81
bin=kmeans;pval=0.0001;clust=Louvain;m=0.3 26225 0.837 0.795
bin=kmeans;pval=0.0001;clust=Louvain;m=0.3 777572 0.833 0.806
bin=kmeans;pval=0.0001;clust=Louvain;m=0.3 288389 0.835 0.804

bin=kmeans;pval=0.0001;clust=Louvain;m=0.4 670487 0.832 0.798
bin=kmeans;pval=0.0001;clust=Louvain;m=0.4 116739 0.838 0.808
bin=kmeans;pval=0.0001;clust=Louvain;m=0.4 26225 0.827 0.808
bin=kmeans;pval=0.0001;clust=Louvain;m=0.4 777572 0.833 0.808
bin=kmeans;pval=0.0001;clust=Louvain;m=0.4 288389 0.835 0.802

bin=kmeans;pval=0.0001;clust=Louvain;m=0.5 670487 0.848 0.816
bin=kmeans;pval=0.

bin=kmeans;pval=0.0005;clust=Louvain;m=0.5 26225 0.85 0.82
bin=kmeans;pval=0.0005;clust=Louvain;m=0.5 777572 0.847 0.813
bin=kmeans;pval=0.0005;clust=Louvain;m=0.5 288389 0.83 0.832

bin=kmeans;pval=0.0005;clust=Louvain;m=0.6 670487 0.852 0.834
bin=kmeans;pval=0.0005;clust=Louvain;m=0.6 116739 0.851 0.806
bin=kmeans;pval=0.0005;clust=Louvain;m=0.6 26225 0.809 0.835
bin=kmeans;pval=0.0005;clust=Louvain;m=0.6 777572 0.815 0.834
bin=kmeans;pval=0.0005;clust=Louvain;m=0.6 288389 0.852 0.833

bin=kmeans;pval=0.0005;clust=Louvain;m=0.7 670487 0.852 0.834
bin=kmeans;pval=0.0005;clust=Louvain;m=0.7 116739 0.85 0.838
bin=kmeans;pval=0.0005;clust=Louvain;m=0.7 26225 0.851 0.835
bin=kmeans;pval=0.0005;clust=Louvain;m=0.7 777572 0.852 0.835
bin=kmeans;pval=0.0005;clust=Louvain;m=0.7 288389 0.851 0.823

bin=kmeans;pval=0.0005;clust=Louvain;m=0.8 670487 0.846 0.835
bin=kmeans;pval=0.0005;clust=Louvain;m=0.8 116739 0.843 0.839
bin=kmeans;pval=0.0005;clust=Louvain;m=0.8 26225 0.845 0.84
bin=kmeans;pva

bin=kmeans;pval=0.005;clust=Louvain;m=0.8 288389 0.77 0.835

bin=kmeans;pval=0.005;clust=Louvain;m=0.9 670487 0.742 0.831
bin=kmeans;pval=0.005;clust=Louvain;m=0.9 116739 0.738 0.831
bin=kmeans;pval=0.005;clust=Louvain;m=0.9 26225 0.843 0.832
bin=kmeans;pval=0.005;clust=Louvain;m=0.9 777572 0.843 0.831
bin=kmeans;pval=0.005;clust=Louvain;m=0.9 288389 0.738 0.83

bin=GMM;pval=0.005;clust=Louvain;m=0 670487 0.819 0.83
bin=GMM;pval=0.005;clust=Louvain;m=0 116739 0.815 0.831
bin=GMM;pval=0.005;clust=Louvain;m=0 26225 0.817 0.83
bin=GMM;pval=0.005;clust=Louvain;m=0 777572 0.783 0.834
bin=GMM;pval=0.005;clust=Louvain;m=0 288389 0.817 0.835

bin=GMM;pval=0.005;clust=Louvain;m=0.3 670487 0.778 0.719
bin=GMM;pval=0.005;clust=Louvain;m=0.3 116739 0.802 0.725
bin=GMM;pval=0.005;clust=Louvain;m=0.3 26225 0.779 0.723
bin=GMM;pval=0.005;clust=Louvain;m=0.3 777572 0.792 0.717
bin=GMM;pval=0.005;clust=Louvain;m=0.3 288389 0.776 0.727

bin=GMM;pval=0.005;clust=Louvain;m=0.4 670487 0.794 0.727
bin=GMM;p

bin=GMM;pval=0.001;clust=Louvain;m=0.4 288389 0.84 0.723

bin=GMM;pval=0.001;clust=Louvain;m=0.5 670487 0.839 0.747
bin=GMM;pval=0.001;clust=Louvain;m=0.5 116739 0.827 0.764
bin=GMM;pval=0.001;clust=Louvain;m=0.5 26225 0.837 0.756
bin=GMM;pval=0.001;clust=Louvain;m=0.5 777572 0.84 0.755
bin=GMM;pval=0.001;clust=Louvain;m=0.5 288389 0.842 0.753

bin=GMM;pval=0.001;clust=Louvain;m=0.6 670487 0.84 0.765
bin=GMM;pval=0.001;clust=Louvain;m=0.6 116739 0.799 0.763
bin=GMM;pval=0.001;clust=Louvain;m=0.6 26225 0.796 0.763
bin=GMM;pval=0.001;clust=Louvain;m=0.6 777572 0.8 0.762
bin=GMM;pval=0.001;clust=Louvain;m=0.6 288389 0.817 0.769

bin=GMM;pval=0.001;clust=Louvain;m=0.7 670487 0.805 0.807
bin=GMM;pval=0.001;clust=Louvain;m=0.7 116739 0.838 0.805
bin=GMM;pval=0.001;clust=Louvain;m=0.7 26225 0.842 0.807
bin=GMM;pval=0.001;clust=Louvain;m=0.7 777572 0.843 0.807
bin=GMM;pval=0.001;clust=Louvain;m=0.7 288389 0.844 0.809

bin=GMM;pval=0.001;clust=Louvain;m=0.8 670487 0.844 0.748
bin=GMM;pval=0.001


bin=GMM;pval=0.01;clust=Louvain;m=0.9 670487 0.815 0.829
bin=GMM;pval=0.01;clust=Louvain;m=0.9 116739 0.816 0.828
bin=GMM;pval=0.01;clust=Louvain;m=0.9 26225 0.814 0.765
bin=GMM;pval=0.01;clust=Louvain;m=0.9 777572 0.817 0.831
bin=GMM;pval=0.01;clust=Louvain;m=0.9 288389 0.817 0.762

bin=ward;pval=0.01;clust=Louvain;m=0 670487 0.803 0.833
bin=ward;pval=0.01;clust=Louvain;m=0 116739 0.803 0.833
bin=ward;pval=0.01;clust=Louvain;m=0 26225 0.803 0.833
bin=ward;pval=0.01;clust=Louvain;m=0 777572 0.729 0.833
bin=ward;pval=0.01;clust=Louvain;m=0 288389 0.801 0.833

bin=ward;pval=0.01;clust=Louvain;m=0.3 670487 0.84 0.829
bin=ward;pval=0.01;clust=Louvain;m=0.3 116739 0.844 0.822
bin=ward;pval=0.01;clust=Louvain;m=0.3 26225 0.773 0.838
bin=ward;pval=0.01;clust=Louvain;m=0.3 777572 0.829 0.818
bin=ward;pval=0.01;clust=Louvain;m=0.3 288389 0.844 0.838

bin=ward;pval=0.01;clust=Louvain;m=0.4 670487 0.856 0.834
bin=ward;pval=0.01;clust=Louvain;m=0.4 116739 0.84 0.828
bin=ward;pval=0.01;clust=Louva

bin=ward;pval=0.05;clust=Louvain;m=0.5 26225 0.851 0.828
bin=ward;pval=0.05;clust=Louvain;m=0.5 777572 0.853 0.829
bin=ward;pval=0.05;clust=Louvain;m=0.5 288389 0.821 0.829

bin=ward;pval=0.05;clust=Louvain;m=0.6 670487 0.854 0.831
bin=ward;pval=0.05;clust=Louvain;m=0.6 116739 0.853 0.831
bin=ward;pval=0.05;clust=Louvain;m=0.6 26225 0.851 0.831
bin=ward;pval=0.05;clust=Louvain;m=0.6 777572 0.855 0.83
bin=ward;pval=0.05;clust=Louvain;m=0.6 288389 0.851 0.83

bin=ward;pval=0.05;clust=Louvain;m=0.7 670487 0.848 0.826
bin=ward;pval=0.05;clust=Louvain;m=0.7 116739 0.849 0.827
bin=ward;pval=0.05;clust=Louvain;m=0.7 26225 0.837 0.826
bin=ward;pval=0.05;clust=Louvain;m=0.7 777572 0.847 0.826
bin=ward;pval=0.05;clust=Louvain;m=0.7 288389 0.837 0.826

bin=ward;pval=0.05;clust=Louvain;m=0.8 670487 0.852 0.812
bin=ward;pval=0.05;clust=Louvain;m=0.8 116739 0.845 0.812
bin=ward;pval=0.05;clust=Louvain;m=0.8 26225 0.845 0.812
bin=ward;pval=0.05;clust=Louvain;m=0.8 777572 0.852 0.813
bin=ward;pval=0.0

In [8]:
out_dir = "results_on_real_data_WGCNA/"
### WGCNA
ds_values = [0,1,2,3,4]
dch = 0.995

for pval in pvals:
    for bin_method in bin_methods:
        for ds in ds_values:
            # save parameters as a ;-separated string
            params = "bin="+bin_method+";pval="+str(pval)
            params += ";clust="+"WGCNA"+";ds="+str(ds)+";dch="+str(dch)
            print()
            for r in range(n_runs):
                seed = seeds[r]
                params_dict = {"parameters":params, "seed":seed,"run":r}
                
                ### running TCGA or reading results
                try:
                    t0 = time()
                    fname = out_dir+basename_t+".seed="+str(seed)+".bin="+bin_method \
                    +",pval="+str(pval)+",clust=WGCNA"+",ds="+str(ds)+",dch="+str(dch)+".biclusters.tsv"
                    result_t = read_bic_table(fname)
                    """result_t = run_DESMOND(exprs_file_t, basename_t, out_dir=out_dir,
                                                save=True, load = True,
                                                ceiling = 3,
                                                min_n_samples = 5,
                                                bin_method = bin_method, pval = pval,
                                                clust_method = "WGCNA",
                                                ds = ds, dch  = dch,
                                                #similarity_cutoffs = similarity_cutoffs,
                                                seed = seed,
                                                verbose = False)"""
                    time_t = time()-t0
                    # find the best matches between TCGA biclusters and subtypes
                    # and calculate overall performance == weighted sum of Jaccard indexes
                    performance_t, bm_dict_t = calculate_perfromance(result_t, known_groups_t,all_samples_t)
                    performance_t = performance_t.to_dict()
                    performance_t.update(params_dict)
                    performance_t["time"] = time_t
                    subt_t.append(performance_t)
                    t_failed = False
                except:
                    print("TCGA biclustering failed with ",seed,  pval,bin_method ,file = sys.stderr)
                    print(fname)
                    t_failed = True
                    subt_t.append({params_dict})
                
                ### running METABRIC or reading results
                try:
                    t0 = time()
                    fname = out_dir+basename_m+".seed="+str(seed)+".bin="+bin_method \
                    +",pval="+str(pval)+",clust=WGCNA"+",ds="+str(ds)+",dch="+str(dch)+".biclusters.tsv"
                    result_m = read_bic_table(fname)
                    """result_m = run_DESMOND(exprs_file_m, basename_m, out_dir=out_dir,
                                                save=True, load = True,
                                                ceiling = 3,
                                                min_n_samples = 5,
                                                bin_method = bin_method, pval = pval,
                                                clust_method = "WGCNA",
                                                ds = ds, dch  = dch,
                                                #similarity_cutoffs = similarity_cutoffs,
                                                seed = seed,
                                                verbose = False)"""
                    time_m = time()-t0
                    # find the best matches between METABRIC biclusters and subtypes
                    # and calculate overall performance == weighted sum of Jaccard indexes
                    performance_m,bm_dict_m = calculate_perfromance(result_m, known_groups_m,all_samples_m)
                    performance_m = performance_m.to_dict()
                    performance_m.update(params_dict)
                    performance_m["time"] = time_m
                    subt_m.append(performance_m)
                    m_failed = False
                except:
                    print("METABRIC biclustering failed with ",seed,  pval,bin_method,file = sys.stderr)
                    print(fname)
                    m_failed = True
                    subt_m.append(params_dict)
                
                print(params,seed, round(performance_t["PAM50"],3),round(performance_m["PAM50"],3))    
                
                # compare clustering results - only if gene sets are defined for each cluster
                if not (t_failed or m_failed): 
                    N = exprs_m.shape[0]
                    clust_sim, bm, bm2 = compare_gene_clusters(result_t,result_m, N)
                    
                    
                else:
                    clust_sim = {}
                clust_sim.update(params_dict)
                clustering_similarities.append(clust_sim)



bin=kmeans;pval=0.0001;clust=WGCNA;ds=0;dch=0.995 670487 0.813 0.851
bin=kmeans;pval=0.0001;clust=WGCNA;ds=0;dch=0.995 116739 0.814 0.851
bin=kmeans;pval=0.0001;clust=WGCNA;ds=0;dch=0.995 26225 0.828 0.844
bin=kmeans;pval=0.0001;clust=WGCNA;ds=0;dch=0.995 777572 0.811 0.848
bin=kmeans;pval=0.0001;clust=WGCNA;ds=0;dch=0.995 288389 0.819 0.851

bin=kmeans;pval=0.0001;clust=WGCNA;ds=1;dch=0.995 670487 0.718 0.837
bin=kmeans;pval=0.0001;clust=WGCNA;ds=1;dch=0.995 116739 0.72 0.838
bin=kmeans;pval=0.0001;clust=WGCNA;ds=1;dch=0.995 26225 0.784 0.841
bin=kmeans;pval=0.0001;clust=WGCNA;ds=1;dch=0.995 777572 0.806 0.838
bin=kmeans;pval=0.0001;clust=WGCNA;ds=1;dch=0.995 288389 0.743 0.836

bin=kmeans;pval=0.0001;clust=WGCNA;ds=2;dch=0.995 670487 0.738 0.839
bin=kmeans;pval=0.0001;clust=WGCNA;ds=2;dch=0.995 116739 0.757 0.842
bin=kmeans;pval=0.0001;clust=WGCNA;ds=2;dch=0.995 26225 0.752 0.842
bin=kmeans;pval=0.0001;clust=WGCNA;ds=2;dch=0.995 777572 0.757 0.839
bin=kmeans;pval=0.0001;clust=WGCNA;

bin=GMM;pval=0.0005;clust=WGCNA;ds=4;dch=0.995 26225 0.783 0.818
bin=GMM;pval=0.0005;clust=WGCNA;ds=4;dch=0.995 777572 0.768 0.803
bin=GMM;pval=0.0005;clust=WGCNA;ds=4;dch=0.995 288389 0.743 0.796

bin=ward;pval=0.0005;clust=WGCNA;ds=0;dch=0.995 670487 0.821 0.8
bin=ward;pval=0.0005;clust=WGCNA;ds=0;dch=0.995 116739 0.835 0.825
bin=ward;pval=0.0005;clust=WGCNA;ds=0;dch=0.995 26225 0.843 0.778
bin=ward;pval=0.0005;clust=WGCNA;ds=0;dch=0.995 777572 0.843 0.833
bin=ward;pval=0.0005;clust=WGCNA;ds=0;dch=0.995 288389 0.837 0.818

bin=ward;pval=0.0005;clust=WGCNA;ds=1;dch=0.995 670487 0.818 0.783
bin=ward;pval=0.0005;clust=WGCNA;ds=1;dch=0.995 116739 0.833 0.82
bin=ward;pval=0.0005;clust=WGCNA;ds=1;dch=0.995 26225 0.754 0.778
bin=ward;pval=0.0005;clust=WGCNA;ds=1;dch=0.995 777572 0.698 0.83
bin=ward;pval=0.0005;clust=WGCNA;ds=1;dch=0.995 288389 0.729 0.819

bin=ward;pval=0.0005;clust=WGCNA;ds=2;dch=0.995 670487 0.748 0.833
bin=ward;pval=0.0005;clust=WGCNA;ds=2;dch=0.995 116739 0.841 0.828
bi

bin=kmeans;pval=0.001;clust=WGCNA;ds=4;dch=0.995 116739 0.81 0.842
bin=kmeans;pval=0.001;clust=WGCNA;ds=4;dch=0.995 26225 0.768 0.84
bin=kmeans;pval=0.001;clust=WGCNA;ds=4;dch=0.995 777572 0.797 0.84
bin=kmeans;pval=0.001;clust=WGCNA;ds=4;dch=0.995 288389 0.785 0.835

bin=GMM;pval=0.001;clust=WGCNA;ds=0;dch=0.995 670487 0.763 0.816
bin=GMM;pval=0.001;clust=WGCNA;ds=0;dch=0.995 116739 0.738 0.817
bin=GMM;pval=0.001;clust=WGCNA;ds=0;dch=0.995 26225 0.766 0.82
bin=GMM;pval=0.001;clust=WGCNA;ds=0;dch=0.995 777572 0.768 0.822
bin=GMM;pval=0.001;clust=WGCNA;ds=0;dch=0.995 288389 0.812 0.809

bin=GMM;pval=0.001;clust=WGCNA;ds=1;dch=0.995 670487 0.786 0.818
bin=GMM;pval=0.001;clust=WGCNA;ds=1;dch=0.995 116739 0.768 0.818
bin=GMM;pval=0.001;clust=WGCNA;ds=1;dch=0.995 26225 0.788 0.82
bin=GMM;pval=0.001;clust=WGCNA;ds=1;dch=0.995 777572 0.784 0.82
bin=GMM;pval=0.001;clust=WGCNA;ds=1;dch=0.995 288389 0.796 0.811

bin=GMM;pval=0.001;clust=WGCNA;ds=2;dch=0.995 670487 0.794 0.821
bin=GMM;pval=0.001;

bin=ward;pval=0.01;clust=WGCNA;ds=4;dch=0.995 26225 0.809 0.84
bin=ward;pval=0.01;clust=WGCNA;ds=4;dch=0.995 777572 0.809 0.841
bin=ward;pval=0.01;clust=WGCNA;ds=4;dch=0.995 288389 0.808 0.807

bin=kmeans;pval=0.05;clust=WGCNA;ds=0;dch=0.995 670487 0.809 0.847
bin=kmeans;pval=0.05;clust=WGCNA;ds=0;dch=0.995 116739 0.807 0.843
bin=kmeans;pval=0.05;clust=WGCNA;ds=0;dch=0.995 26225 0.815 0.843
bin=kmeans;pval=0.05;clust=WGCNA;ds=0;dch=0.995 777572 0.783 0.852
bin=kmeans;pval=0.05;clust=WGCNA;ds=0;dch=0.995 288389 0.794 0.844

bin=kmeans;pval=0.05;clust=WGCNA;ds=1;dch=0.995 670487 0.806 0.848
bin=kmeans;pval=0.05;clust=WGCNA;ds=1;dch=0.995 116739 0.807 0.843
bin=kmeans;pval=0.05;clust=WGCNA;ds=1;dch=0.995 26225 0.825 0.843
bin=kmeans;pval=0.05;clust=WGCNA;ds=1;dch=0.995 777572 0.779 0.846
bin=kmeans;pval=0.05;clust=WGCNA;ds=1;dch=0.995 288389 0.794 0.844

bin=kmeans;pval=0.05;clust=WGCNA;ds=2;dch=0.995 670487 0.794 0.845
bin=kmeans;pval=0.05;clust=WGCNA;ds=2;dch=0.995 116739 0.807 0.845
bi

In [9]:
cl = "Intrinsic"
df = pd.DataFrame.from_records(subt_t).groupby("parameters").agg("mean").sort_values(cl,ascending = False)
df.head(10)

,PAM50,Intrinsic,PAM50_AB,SCMOD2,IHC,Luminal,Basal,Her2,LumA,LumB,...,Claudin-low,IHC_HER2,IHC_ER,IHC_PR,IHC_TNBC,NET_kmeans,NET_ward,seed,run,time
parameters,,,,,,,,,,,,,,,,,,,,,
bin=ward;pval=0.0005;clust=Louvain;m=0.9,0.857574,0.826647,0.613004,0.617021,0.732630,0.920398,0.950249,0.480769,0.673546,0.431694,...,0.125581,0.423469,0.865407,0.785455,0.466981,0.445652,0.380952,375882.4,2.0,0.025240
bin=ward;pval=0.005;clust=Louvain;m=0.6,0.857445,0.823116,0.581036,0.566346,0.700372,0.918184,0.953000,0.492647,0.580247,0.464988,...,0.121156,0.394444,0.863274,0.782837,0.468127,0.682540,0.561404,375882.4,2.0,0.025155
bin=ward;pval=0.0005;clust=Louvain;m=0.6,0.854990,0.821270,0.607978,0.618957,0.699794,0.918388,0.944724,0.478261,0.641405,0.457088,...,0.117925,0.392265,0.863587,0.785953,0.471154,0.682540,0.561404,375882.4,2.0,0.026506
bin=ward;pval=0.0001;clust=Louvain;m=0.7,0.855166,0.820929,0.563368,0.575442,0.700871,0.916049,0.949749,0.492647,0.603748,0.410673,...,0.117371,0.394444,0.866040,0.786490,0.476463,0.457447,0.363636,375882.4,2.0,0.024909
bin=kmeans;pval=0.0001;clust=Louvain;m=0.6,0.851945,0.820355,0.594102,0.564397,0.709699,0.918403,0.937576,0.477803,0.606481,0.464504,...,0.118838,0.420287,0.867627,0.788224,0.469644,0.099448,0.069686,375882.4,2.0,0.023389
bin=ward;pval=0.05;clust=Louvain;m=0.6,0.853010,0.818859,0.613107,0.652709,0.701071,0.914730,0.945455,0.492647,0.703397,0.465157,...,0.118935,0.394444,0.862983,0.784213,0.475269,0.457447,0.363636,375882.4,2.0,0.028532
bin=ward;pval=0.001;clust=Louvain;m=0.8,0.848993,0.818411,0.607264,0.629202,0.699172,0.908106,0.946191,0.492647,0.673546,0.410673,...,0.121450,0.394444,0.863347,0.784103,0.471529,0.445652,0.380952,375882.4,2.0,0.029343
bin=kmeans;pval=0.001;clust=Louvain;m=0.7,0.851621,0.817525,0.581618,0.576748,0.713522,0.914608,0.938077,0.477803,0.619340,0.480190,...,0.118415,0.420287,0.870372,0.786821,0.476031,0.101228,0.070544,375882.4,2.0,0.028658
bin=kmeans;pval=0.0005;clust=Louvain;m=0.7,0.851183,0.817105,0.593678,0.565733,0.727308,0.914385,0.936879,0.477803,0.617115,0.479489,...,0.118148,0.420287,0.871082,0.786680,0.479190,0.104493,0.073613,375882.4,2.0,0.026505


In [10]:
df2 = pd.DataFrame.from_records(subt_m).groupby("parameters").agg("mean").sort_values(cl,ascending = False)
df2.head(10)

,PAM50,Intrinsic,PAM50_AB,SCMOD2,IHC,Luminal,Basal,Her2,LumA,LumB,...,Claudin-low,IHC_HER2,IHC_ER,IHC_PR,IHC_TNBC,NET_kmeans,NET_ward,seed,run,time
parameters,,,,,,,,,,,,,,,,,,,,,
bin=kmeans;pval=0.0001;clust=WGCNA;ds=0;dch=0.995,0.848874,0.816334,0.546904,0.541967,0.732804,0.930145,0.849441,0.563183,0.466814,0.546428,...,0.162624,0.680394,0.929429,0.677249,0.676319,0.702728,0.491692,375882.4,2.0,0.025176
bin=kmeans;pval=0.05;clust=WGCNA;ds=0;dch=0.995,0.845809,0.814377,0.580324,0.609019,0.752801,0.930618,0.859728,0.527241,0.577181,0.547714,...,0.159904,0.755871,0.934630,0.680325,0.683256,0.703458,0.494427,375882.4,2.0,0.034726
bin=kmeans;pval=0.0005;clust=WGCNA;ds=0;dch=0.995,0.845574,0.814017,0.547887,0.554153,0.739486,0.930028,0.855674,0.532637,0.480255,0.546121,...,0.161102,0.746905,0.929310,0.677906,0.679674,0.712920,0.500952,375882.4,2.0,0.027904
bin=kmeans;pval=0.05;clust=WGCNA;ds=1;dch=0.995,0.844696,0.813322,0.578625,0.611432,0.754583,0.930618,0.859728,0.517938,0.577181,0.546451,...,0.159904,0.778546,0.934630,0.680325,0.683256,0.703458,0.494427,375882.4,2.0,0.037259
bin=kmeans;pval=0.05;clust=WGCNA;ds=2;dch=0.995,0.844550,0.813164,0.571613,0.611757,0.759172,0.933209,0.862203,0.497880,0.577181,0.544383,...,0.159904,0.835699,0.938716,0.681195,0.683256,0.703458,0.494427,375882.4,2.0,0.041913
bin=kmeans;pval=0.005;clust=WGCNA;ds=0;dch=0.995,0.844284,0.812621,0.554029,0.562151,0.742437,0.929955,0.855456,0.523139,0.500634,0.546688,...,0.161526,0.742671,0.927927,0.677114,0.679299,0.708828,0.498131,375882.4,2.0,0.030412
bin=kmeans;pval=0.001;clust=WGCNA;ds=0;dch=0.995,0.843128,0.811624,0.545513,0.558938,0.742343,0.929714,0.855674,0.514266,0.488602,0.539539,...,0.161102,0.776353,0.928745,0.677015,0.679674,0.711538,0.500000,375882.4,2.0,0.031464
bin=kmeans;pval=0.05;clust=WGCNA;ds=3;dch=0.995,0.842337,0.810977,0.577097,0.616402,0.757199,0.930759,0.859728,0.494035,0.577181,0.549503,...,0.159904,0.838214,0.941056,0.680648,0.683256,0.706143,0.496279,375882.4,2.0,0.073764
bin=kmeans;pval=0.005;clust=WGCNA;ds=1;dch=0.995,0.842155,0.810588,0.551286,0.562616,0.746993,0.929955,0.855456,0.506131,0.498434,0.544393,...,0.161526,0.773042,0.927927,0.677114,0.679299,0.708828,0.498131,375882.4,2.0,0.036980


In [28]:
params = "bin=kmeans;pval=0.05;clust=WGCNA;ds=3;dch=0.995"
df.loc[params,:]

PAM50               0.839289
Intrinsic           0.808836
PAM50_AB            0.641686
SCMOD2              0.668707
IHC                 0.710369
Luminal             0.899294
Basal               0.939083
Her2                0.472495
LumA                0.717384
LumB                0.474619
Normal              0.089188
Claudin-low         0.121469
IHC_HER2            0.415469
IHC_ER              0.860178
IHC_PR              0.780174
IHC_TNBC            0.468461
NET_kmeans          0.227089
NET_ward            0.187310
seed           375882.400000
run                 2.000000
time                0.056577
rank               39.000000
Name: bin=kmeans;pval=0.05;clust=WGCNA;ds=3;dch=0.995, dtype: float64

In [29]:
df2.loc[params,:]

PAM50               0.842337
Intrinsic           0.810977
PAM50_AB            0.577097
SCMOD2              0.616402
IHC                 0.757199
Luminal             0.930759
Basal               0.859728
Her2                0.494035
LumA                0.577181
LumB                0.549503
Normal              0.107918
Claudin-low         0.159904
IHC_HER2            0.838214
IHC_ER              0.941056
IHC_PR              0.680648
IHC_TNBC            0.683256
NET_kmeans          0.706143
NET_ward            0.496279
seed           375882.400000
run                 2.000000
time                0.073764
rank                7.000000
Name: bin=kmeans;pval=0.05;clust=WGCNA;ds=3;dch=0.995, dtype: float64

In [21]:
pd.DataFrame.from_records(clustering_similarities).to_csv("UnPaSt_similarities.tsv",sep = "\t")
pd.DataFrame.from_records(subt_t).to_csv("UnPaSt_TCGA.tsv",sep = "\t")
pd.DataFrame.from_records(subt_m).to_csv("UnPaSt_METABRIC.tsv",sep = "\t")


In [22]:
#df = pd.read_csv("UnPaSt_TCGA.tsv",sep = "\t",index_col =0).groupby("parameters").agg("mean").sort_values(by= "overall_performance_Intrinsic",ascending=False)
#df2 = pd.read_csv("UnPaSt_METABRIC.tsv",sep = "\t",index_col =0).groupby("parameters").agg("mean").sort_values(by= "overall_performance_Intrinsic",ascending=False)


In [30]:
df["rank"] = range(df.shape[0])
df2["rank"] = range(df2.shape[0])
r = df["rank"]+df2["rank"]
r.sort_values()

parameters
bin=kmeans;pval=0.05;clust=WGCNA;ds=3;dch=0.995     46
bin=ward;pval=0.005;clust=Louvain;m=0.5             52
bin=kmeans;pval=0.0005;clust=Louvain;m=0            53
bin=kmeans;pval=0.0001;clust=Louvain;m=0.7          55
bin=kmeans;pval=0.0001;clust=Louvain;m=0.6          61
                                                  ... 
bin=GMM;pval=0.01;clust=Louvain;m=0.3              423
bin=GMM;pval=0.005;clust=Louvain;m=0.3             427
bin=GMM;pval=0.05;clust=Louvain;m=0.4              433
bin=GMM;pval=0.0005;clust=Louvain;m=0.3            452
bin=GMM;pval=0.001;clust=Louvain;m=0.3             463
Name: rank, Length: 234, dtype: int64

In [26]:
#s = pd.read_csv("UnPaSt_similarities.tsv",sep = "\t",index_col=0).groupby("parameters").agg("mean")
s = pd.DataFrame.from_records(clustering_similarities).groupby("parameters").agg("mean")
s["avg_percent_matched"] = (s["percent_matched_1"]+s["percent_matched_2"])*0.5
s.sort_values(by = "avg_percent_matched",ascending = False)

,n_1,n_2,percent_matched_1,percent_matched_2,n_shared_genes_1,avg_bm_J_1,n_shared_genes_2,avg_bm_J_2,seed,run,avg_percent_matched
parameters,,,,,,,,,,,
bin=kmeans;pval=0.005;clust=WGCNA;ds=0;dch=0.995,46.4,46.6,0.593624,0.597131,318.6,0.158151,316.8,0.154152,375882.4,2.0,0.595378
bin=kmeans;pval=0.01;clust=WGCNA;ds=0;dch=0.995,41.2,48.8,0.626871,0.562146,353.0,0.149535,345.2,0.145801,375882.4,2.0,0.594509
bin=kmeans;pval=0.0001;clust=WGCNA;ds=0;dch=0.995,32.8,39.4,0.580117,0.574017,207.0,0.104053,227.8,0.093804,375882.4,2.0,0.577067
bin=kmeans;pval=0.01;clust=WGCNA;ds=1;dch=0.995,59.4,62.8,0.562173,0.566995,359.8,0.186318,355.2,0.178967,375882.4,2.0,0.564584
bin=GMM;pval=0.0001;clust=WGCNA;ds=0;dch=0.995,21.8,42.2,0.714585,0.411483,194.2,0.144440,176.2,0.132974,375882.4,2.0,0.563034
...,...,...,...,...,...,...,...,...,...,...,...
bin=ward;pval=0.001;clust=WGCNA;ds=4;dch=0.995,311.6,251.6,0.149631,0.221098,198.2,0.254058,249.2,0.217067,375882.4,2.0,0.185365
bin=kmeans;pval=0.001;clust=Louvain;m=0,61.6,40.8,0.149111,0.221215,30.6,0.353763,29.2,0.399546,375882.4,2.0,0.185163
bin=GMM;pval=0.01;clust=Louvain;m=0,76.6,47.6,0.112605,0.256472,34.2,0.367585,50.4,0.306471,375882.4,2.0,0.184539


In [27]:
s.loc[bic_id,:]

n_1                       163.800000
n_2                       164.400000
percent_matched_1           0.399449
percent_matched_2           0.335696
n_shared_genes_1          279.200000
avg_bm_J_1                  0.195482
n_shared_genes_2          282.800000
avg_bm_J_2                  0.224721
seed                   375882.400000
run                         2.000000
avg_percent_matched         0.367572
Name: bin=kmeans;pval=0.05;clust=WGCNA;ds=3;dch=0.995, dtype: float64